In [1]:
import pandas as pd
import networkx as nx
from tqdm import tqdm

In [ ]:
%%time
dftemp=pd.read_csv('../PushshiftDumps/scripts/RC_2022-04.csv', lineterminator='\n',index_col=0)

In [ ]:
dftemp.reset_index(inplace=True)

In [ ]:
%%time
dftemp['id\r']=dftemp['id\r'].str.replace('\r','')

In [ ]:
dftemp=dftemp.rename(columns={'id\r':'id'})

In [ ]:
head=list(dftemp.columns)

In [ ]:
head[-1]='id'

In [ ]:
dftemp.columns=head

In [ ]:
dftemp

In [ ]:
%%time
def process(df):
    df=df[df['body']!='[deleted]']
    
    df=df[df['body']!='[removed]']
    
    df=df[df['author']!='[removed]']
    
    df=df[df['author']!='[deleted]']
    
    df=df[df['created_utc']!='created_utc']
    return df

import re
tqdm.pandas(desc='bar')
def remove_pre(_x):
    try:
        _match = re.search(r't[1]_(\w+)', _x)
        _result = _match.group(1)

        return _result
    except:
        return ''



In [ ]:
%%time
dftemp=process(dftemp)

In [ ]:
%%time
df1=dftemp[dftemp['parent_id'].str.startswith('t1')]

In [ ]:
df1['parent_id']=df1['parent_id'].progress_apply(lambda x: remove_pre(x))

In [ ]:
df1.to_csv('df1_t1_only.csv',index=None)

In [83]:
import pandas as pd
df1=pd.read_csv('df1_t1_only.csv',lineterminator='\n')

In [4]:
%%time
df1['date'] = pd.to_datetime(df1['created_utc'], unit='s')

CPU times: user 1.44 s, sys: 457 ms, total: 1.89 s
Wall time: 1.89 s


In [5]:
ndf=df1[['author','parent_id',]].drop_duplicates()

In [88]:
#df2=pd.read_csv('rc_df2.csv')

In [94]:
dic=dict(zip(df1.id,df1.author))

In [120]:
import networkx as nx
def reply_network(bot):
    if lang=='en':
        bot['Class']=bot['stance']+' '+bot['class']
        dic_Class = dict(zip(bot.Id,bot.Class))
        df3['id_Class']=df3['username'].map(dic_Class)
        

    if lang=='de':
        dic_Class=dict(zip(bot.author,bot.Class))
        df3['id_Class']=df3['author'].map(dic_Class)
        
    
    df3['be_replied_id_Class']=df3['be_replid_user'].map(dic_Class)
    
    
        #df3['be_replied_user_id']=df3[''
        
        
    return df3

In [140]:
lang='de'
if lang=='en':
    df2=pd.read_csv('test_Bucha_en.csv')
    
    df2['be_replid_user']=df2['reply_id'].map(dic)
    
    df3=df2[df2['be_replid_user'].notna()]
    bot=pd.read_csv('test_label_Bucha_en.csv')#en
elif lang=='de':
    df2=pd.read_csv('test_Bucha_de.csv')
    
    df2['be_replid_user']=df2['parent_id'].map(dic)
    
    df3=df2[df2['be_replid_user'].notna()]
    bot=pd.read_csv('label_file_de.csv')

In [97]:
#bot=pd.read_csv('botdf_id_gexf_human_bot_plus_autolabel.csv')

In [98]:
#bot=pd.read_csv('test_label_Bucha_en.csv')#en

In [72]:
#bot=pd.read_csv('label_file_de.csv')#de

In [74]:
'''def reply_network(bot):
    if lang=='en':
        bot['Class']=bot['stance']+' '+bot['class']
        dic_Class = dict(zip(author.Id,bot.Class))
        

    if lang=='de':
        dic_Class=dict(zip(bot.author,bot.Class))
    df3['id_Class']=df3['username'].map(dic_Class)
        
    
    df3['be_replied_id_Class']=df3['be_replid_user'].map(dic_Class)
    
    
        #df3['be_replied_user_id']=df3[''
        
        
    return df3'''

In [141]:
#lang='de'
df3=reply_network(bot)
if lang=='en':
    user='usernanme'
elif lang=='de':
    user='author'
G=nx.from_pandas_edgelist(df3,user,'be_replid_user',create_using=nx.DiGraph())

/tmp/ipykernel_3454936/4138362349.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['id_Class']=df3['author'].map(dic_Class)
/tmp/ipykernel_3454936/4138362349.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['be_replied_id_Class']=df3['be_replid_user'].map(dic_Class)


In [142]:
nx.write_gexf(G,'test_'+lang+'_reply.gexf')

nx.read_gexf('test_'+lang+'_reply.gexf')

In [125]:
#temp=df3[[user,'id_Class']]
#temp.columns=['Id','Class']
#temp.to_csv('test_'+lang+'_reply.csv',index=None)

In [144]:
#lang='de'
if lang=='en':
    d=pd.read_csv('test_label_Bucha_en.csv')
    d=d[['author','stance','class']]
    d.columns=['Id','stance','class']
    d.to_csv('test_'+lang+'_reply.csv',index=None)
elif lang=='de':
    d=pd.read_csv('label_file_de.csv')
    d=d[['author','label_label','class']]
    d.columns=['Id','stance','class']
    d.to_csv('test_'+lang+'_reply.csv',index=None)
    